In [210]:
import mxnet as mx
import os
from gluoncv.model_zoo import get_model
from gluoncv.utils import viz
from mxnet import gluon
from mxnet.gluon.data.vision import transforms

In [2]:
fine_tune_net = get_model('xception', pretrained=True)

In [3]:
net = gluon.nn.HybridSequential()
with net.name_scope():
    net.add(fine_tune_net.conv1)
    net.add(fine_tune_net.bn1)
    net.add(fine_tune_net.relu)
    net.add(fine_tune_net.conv2)
    net.add(fine_tune_net.bn2)
    net.add(fine_tune_net.relu)
    net.add(fine_tune_net.block1)
    net.add(fine_tune_net.relu)
    net.add(fine_tune_net.block2)
    net.add(fine_tune_net.block3)
    net.add(fine_tune_net.midflow)
    net.add(fine_tune_net.block20)
    net.add(fine_tune_net.relu)
    net.add(fine_tune_net.conv3)
    net.add(fine_tune_net.bn3)
    net.add(fine_tune_net.relu)
    net.add(fine_tune_net.conv4)
    net.add(fine_tune_net.bn4)
    net.add(fine_tune_net.relu)
    net.add(fine_tune_net.conv5)
    net.add(fine_tune_net.bn5)
    net.add(fine_tune_net.relu)
    net.add(fine_tune_net.avgpool)
    net.add(fine_tune_net.flat)
    net.add(gluon.nn.Dropout(0.5))
    net.add(gluon.nn.Dense(25))
    net[-1].collect_params().initialize(init=mx.init.Xavier(),ctx=mx.gpu())
net.hybridize()
net = net.collect_params().reset_ctx(ctx=mx.gpu())

In [97]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [96]:
trainer = gluon.Trainer(net.collect_params(),"Adagrad",{"learning_rate":0.01})

In [98]:
def acc(output,label):
    return (output.argmax(axis=1) == label.astype("float32")).mean().asscalar()

In [236]:
net.collect_params()

hybridsequential9_ (
  Parameter xception654_conv0_weight (shape=(32, 3, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm0_gamma (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm0_beta (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm0_running_mean (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm0_running_var (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter xception654_conv1_weight (shape=(64, 32, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm1_gamma (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm1_beta (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm1_running_mean (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter xception654_batchnorm1_running_var (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter xception654_conv2_weight (shape=(128, 64, 1, 1), dtype=<class 'numpy.float

In [232]:
transformer = transforms.Compose([transforms.Resize(299)])

In [233]:
train_data = gluon.data.vision.ImageRecordDataset('img_file/train.rec',)

In [234]:
train_loader = gluon.data.DataLoader(train_data.transform_first(transformer), batch_size=8,num_workers = 4)

In [241]:
for imgs, labels in train_loader:
    print(labels.as_in_context(mx.gpu()))
    break


[23. 20. 19. 24. 16. 12. 10. 24.  5.  3.  4.  3. 17. 12. 23. 17.]
<NDArray 16 @gpu(0)>


In [163]:
y

23.0